In [1]:
import numpy as np

def optimal_bins_dp(x, y, k):
    N = len(x)
    # Precompute prefix sums for fast MSE calc
    prefix_sum = np.cumsum(np.concatenate([[0], y]))
    prefix_sq = np.cumsum(np.concatenate([[0], y**2]))
    # dp[b][i] = minimum SSE (sum of squared errors) for binning first i points into b bins
    dp = np.full((k+1, N+1), np.inf)
    prev_cut = np.zeros((k+1, N+1), dtype=int)  # to store optimal cut index
    dp[0][0] = 0  # 0 bins for 0 points = 0 error

    # Base case: 1 bin for first i points = error of one segment [1..i]
    for i in range(1, N+1):
        # SSE for segment 1..i:
        sum_y = prefix_sum[i] - prefix_sum[0]
        sum_y2 = prefix_sq[i] - prefix_sq[0]
        seg_len = i
        dp[1][i] = sum_y2 - (sum_y**2)/seg_len

    # Fill DP for b = 2..k
    for b in range(2, k+1):
        for i in range(b, N+1):  # need at least b points for b bins
            # try placing the (b-th) bin start at j (meaning previous cut at j)
            for j in range(b-1, i):
                # error = dp[b-1][j] + SSE of segment (j+1 .. i)
                sum_y = prefix_sum[i] - prefix_sum[j]
                sum_y2 = prefix_sq[i] - prefix_sq[j]
                seg_len = i - j
                sse_segment = sum_y2 - (sum_y**2)/seg_len
                total_sse = dp[b-1][j] + sse_segment
                if total_sse < dp[b][i]:
                    dp[b][i] = total_sse
                    prev_cut[b][i] = j
    # Backtrack to find cut indices
    cuts = []
    b, i = k, N
    while b > 1:
        j = prev_cut[b][i]
        cuts.append(j)
        i, b = j, b-1
    cuts = sorted(cuts)  # indices where bins end
    # Convert cut indices to bin edge values (midpoints between x[c] and x[c+1])
    edges = [ (x[c-1] + x[c])/2 for c in cuts ]
    return edges


In [2]:
from sklearn.cluster import KMeans

def kmeans_binning(x, y, n_bins):
    km = KMeans(n_clusters=n_bins, n_init=10, random_state=0)
    labels = km.fit_predict(x.reshape(-1,1))
    centers = np.sort(km.cluster_centers_.flatten())
    # Compute midpoints between sorted cluster centers as edges
    edges = [(centers[i] + centers[i+1]) / 2 for i in range(len(centers)-1)]
    return edges

edges = kmeans_binning(x_values, y_target, n_bins=5)
print("K-means bin edges:", edges)

NameError: name 'x_values' is not defined

In [3]:
from sklearn.isotonic import IsotonicRegression

def isotonic_binning(x, y, n_bins):
    iso = IsotonicRegression(increasing=True).fit(x, y)
    y_iso_pred = iso.predict(x)           # isotonic fitted values (monotonic sequence)
    # Cluster the isotonic predictions into k groups (e.g., using k-means in value-space)
    centers, labels = KMeans(n_clusters=n_bins, n_init=10, random_state=0) \
                        .fit_predict(y_iso_pred.reshape(-1,1), return_centers=True)
    # Ensure labels correspond to sorted order of centers
    sorted_centers = sorted(enumerate(centers), key=lambda c: c[1])
    label_map = {old: new for new, (old, _) in enumerate(sorted_centers)}
    labels = np.array([ label_map[l] for l in labels ])
    # Identify bin edges where label changes
    edges = []
    for i in range(1, len(x)):
        if labels[i] != labels[i-1]:
            edges.append((x[i-1] + x[i]) / 2)
    return edges

In [4]:
def quantile_supervised_binning(x, y, n_bins):
    # Initial equal-frequency cuts
    sorted_idx = np.argsort(x)
    x_sorted, y_sorted = x[sorted_idx], y[sorted_idx]
    initial_edges = [x_sorted[int(i*len(x)/n_bins)] for i in range(1, n_bins)]
    edges = initial_edges[:]
    improved = True
    while improved:
        improved = False
        # Check each boundary for potential move
        for bi in range(len(edges)):
            # compute current MSE
            mse_current = compute_mse(x, y, edges)
            # Try shifting this edge left or right by one position (if possible) and compute MSE
            new_edges_left = adjust_edge_left(edges, bi, x_sorted)
            new_edges_right = adjust_edge_right(edges, bi, x_sorted)
            # (compute_mse and adjust_edge_left/right are helper functions not shown for brevity)
            best_option = min((edges, mse_current), (new_edges_left, mse_left), (new_edges_right, mse_right), key=lambda t: t[1])
            if best_option[1] < mse_current:
                edges = best_option[0]
                improved = True
                break  # re-start loop after a successful adjustment
    return edges

In [5]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.cluster import KMeans
from sklearn.isotonic import IsotonicRegression
from scipy.stats import norm
import matplotlib.pyplot as plt

# --- Simulate Data ---
def simulate_data(n=1000, scenario="non_monotonic"):
    np.random.seed(42)
    x = np.sort(np.random.uniform(0, 100, n))  # Covariate

    if scenario == "non_monotonic":
        y = np.piecewise(x, [x < 10, (x >= 10) & (x < 40), (x >= 40) & (x < 50), (x >= 50) & (x < 90), x >= 90],
                         [50, 20, 40, 10, 30]) + np.random.normal(0, 5, n)
    elif scenario == "monotonic":
        y = 3 * x + np.random.normal(0, 20, n)
    else:
        raise ValueError("Unknown scenario")

    return x, y

# --- Decision Tree-Based Binning ---
def tree_binning(x, y, n_bins):
    tree = DecisionTreeRegressor(max_leaf_nodes=n_bins, random_state=0)
    tree.fit(x.reshape(-1, 1), y)
    thr = tree.tree_.threshold
    edges = sorted(thr[thr > -2])  # Extract split thresholds
    return edges

# --- Dynamic Programming Optimal Binning ---
def optimal_bins_dp(x, y, k):
    x, y = np.sort(x), y[np.argsort(x)]
    N = len(x)
    prefix_sum = np.cumsum(np.concatenate([[0], y]))
    prefix_sq = np.cumsum(np.concatenate([[0], y**2]))
    dp = np.full((k+1, N+1), np.inf)
    prev_cut = np.zeros((k+1, N+1), dtype=int)
    dp[0][0] = 0

    for i in range(1, N+1):
        sum_y, sum_y2 = prefix_sum[i], prefix_sq[i]
        seg_len = i
        dp[1][i] = sum_y2 - (sum_y**2)/seg_len

    for b in range(2, k+1):
        for i in range(b, N+1):
            for j in range(b-1, i):
                sum_y = prefix_sum[i] - prefix_sum[j]
                sum_y2 = prefix_sq[i] - prefix_sq[j]
                seg_len = i - j
                sse_segment = sum_y2 - (sum_y**2)/seg_len
                total_sse = dp[b-1][j] + sse_segment
                if total_sse < dp[b][i]:
                    dp[b][i] = total_sse
                    prev_cut[b][i] = j

    cuts = []
    b, i = k, N
    while b > 1:
        j = prev_cut[b][i]
        cuts.append(j)
        i, b = j, b-1
    edges = [(x[c-1] + x[c])/2 for c in sorted(cuts)]
    return edges

# --- K-Means Binning ---
def kmeans_binning(x, y, n_bins):
    km = KMeans(n_clusters=n_bins, n_init=10, random_state=0)
    km.fit(x.reshape(-1, 1))
    centers = np.sort(km.cluster_centers_.flatten())
    edges = [(centers[i] + centers[i+1]) / 2 for i in range(len(centers)-1)]
    return edges

# --- Isotonic Regression-Based Binning ---
def isotonic_binning(x, y, n_bins):
    iso = IsotonicRegression(increasing=True).fit(x, y)
    y_iso_pred = iso.predict(x)
    km = KMeans(n_clusters=n_bins, n_init=10, random_state=0)
    labels = km.fit_predict(y_iso_pred.reshape(-1, 1))
    edges = []
    for i in range(1, len(x)):
        if labels[i] != labels[i-1]:
            edges.append((x[i-1] + x[i]) / 2)
    return edges

# --- Quantile-Based Supervised Binning ---
def quantile_supervised_binning(x, y, n_bins):
    sorted_idx = np.argsort(x)
    x_sorted, y_sorted = x[sorted_idx], y[sorted_idx]
    initial_edges = [x_sorted[int(i*len(x)/n_bins)] for i in range(1, n_bins)]
    edges = initial_edges[:]
    improved = True
    while improved:
        improved = False
        for bi in range(len(edges)):
            mse_current = np.mean((y_sorted - np.digitize(x_sorted, edges))**2)
            new_edges_left = edges[:]
            if bi > 0:
                new_edges_left[bi] = (edges[bi-1] + edges[bi]) / 2
            mse_left = np.mean((y_sorted - np.digitize(x_sorted, new_edges_left))**2)
            new_edges_right = edges[:]
            if bi < len(edges)-1:
                new_edges_right[bi] = (edges[bi] + edges[bi+1]) / 2
            mse_right = np.mean((y_sorted - np.digitize(x_sorted, new_edges_right))**2)
            best_option = min((edges, mse_current), (new_edges_left, mse_left), (new_edges_right, mse_right), key=lambda t: t[1])
            if best_option[1] < mse_current:
                edges = best_option[0]
                improved = True
                break
    return edges

# --- Evaluation ---
def evaluate_methods(scenario="non_monotonic", n_bins=5):
    x, y = simulate_data(scenario=scenario)
    methods = {
        "Decision Tree": tree_binning(x, y, n_bins),
        "Dynamic Programming": optimal_bins_dp(x, y, n_bins),
        "K-Means": kmeans_binning(x, y, n_bins),
        "Isotonic Regression": isotonic_binning(x, y, n_bins),
        "Quantile Supervised": quantile_supervised_binning(x, y, n_bins),
    }
    
    results = []
    for method, edges in methods.items():
        bin_means = np.zeros_like(y)
        bin_indices = np.digitize(x, edges)
        for i in range(n_bins):
            bin_means[bin_indices == i] = np.mean(y[bin_indices == i])
        mse = np.mean((y - bin_means) ** 2)
        results.append({"Method": method, "Scenario": scenario, "Bins": n_bins, "MSE": mse})
    
    return pd.DataFrame(results)

# Run evaluation for both scenarios
df_results = pd.concat([evaluate_methods("non_monotonic"), evaluate_methods("monotonic")])
df_results

/Users/soodoku/opt/anaconda3/envs/py311ds/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/soodoku/opt/anaconda3/envs/py311ds/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/soodoku/opt/anaconda3/envs/py311ds/lib/python3.11/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/soodoku/opt/anaconda3/envs/py311ds/lib/python3.11/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Method,Scenario,Bins,MSE
0,Decision Tree,non_monotonic,5,24.403682
1,Dynamic Programming,non_monotonic,5,24.403682
2,K-Means,non_monotonic,5,136.895532
3,Isotonic Regression,non_monotonic,5,195.468801
4,Quantile Supervised,non_monotonic,5,150.475030
0,Decision Tree,monotonic,5,702.929067
1,Dynamic Programming,monotonic,5,610.034663
2,K-Means,monotonic,5,666.676728
3,Isotonic Regression,monotonic,5,611.416474
4,Quantile Supervised,monotonic,5,4413.963329


In [8]:
# Modified evaluation function to handle empty bins
def evaluate_methods_fixed(scenario="non_monotonic", n_bins=5):
    x, y = simulate_data(scenario=scenario)
    methods = {
        "Decision Tree": tree_binning(x, y, n_bins),
        "Dynamic Programming": optimal_bins_dp(x, y, n_bins),
        "K-Means": kmeans_binning(x, y, n_bins),
        "Isotonic Regression": isotonic_binning(x, y, n_bins),
        "Quantile Supervised": quantile_supervised_binning(x, y, n_bins),
    }
    
    results = []
    for method, edges in methods.items():
        bin_means = np.zeros_like(y)
        bin_indices = np.digitize(x, edges)

        # Ensure bins are assigned correctly and handle empty bins
        for i in range(n_bins):
            mask = bin_indices == i
            if np.any(mask):  # If bin is not empty
                bin_means[mask] = np.mean(y[mask])
            else:  # Assign nearest valid bin mean
                valid_bins = np.unique(bin_indices)
                nearest_bin = min(valid_bins, key=lambda v: abs(v - i))  # Find nearest bin index
                bin_means[bin_indices == i] = np.mean(y[bin_indices == nearest_bin])

        mse = np.mean((y - bin_means) ** 2)
        results.append({"Method": method, "Scenario": scenario, "Bins": n_bins, "MSE": mse})
    
    return pd.DataFrame(results)

# Run evaluation with fixed handling of empty bins
df_results_fixed = pd.concat([
    evaluate_methods_fixed("non_monotonic"),
    evaluate_methods_fixed("monotonic")
])

In [9]:
df_results_fixed.groupby("Method")["MSE"].mean().reset_index()

,Method,MSE
0,Decision Tree,363.666374
1,Dynamic Programming,317.219172
2,Isotonic Regression,403.442637
3,K-Means,401.786130
4,Quantile Supervised,2282.219179


In [10]:
from sklearn.metrics import mean_absolute_error

# Huber loss function
def huber_loss(y_true, y_pred, delta=1.0):
    residual = np.abs(y_true - y_pred)
    loss = np.where(residual <= delta, 0.5 * residual ** 2, delta * (residual - 0.5 * delta))
    return np.mean(loss)

# Updated evaluation function with Huber loss optimization
def evaluate_methods_huber(scenario="non_monotonic", n_bins=5, delta=1.0):
    x, y = simulate_data(scenario=scenario)
    methods = {
        "Decision Tree": tree_binning(x, y, n_bins),
        "Dynamic Programming": optimal_bins_dp(x, y, n_bins),
        "K-Means": kmeans_binning(x, y, n_bins),
        "Isotonic Regression": isotonic_binning(x, y, n_bins),
        "Quantile Supervised": quantile_supervised_binning(x, y, n_bins),
    }
    
    results = []
    for method, edges in methods.items():
        bin_means = np.zeros_like(y)
        bin_indices = np.digitize(x, edges)

        # Ensure bins are assigned correctly and handle empty bins
        for i in range(n_bins):
            mask = bin_indices == i
            if np.any(mask):  # If bin is not empty
                bin_means[mask] = np.mean(y[mask])
            else:  # Assign nearest valid bin mean
                valid_bins = np.unique(bin_indices)
                nearest_bin = min(valid_bins, key=lambda v: abs(v - i))  # Find nearest bin index
                bin_means[bin_indices == i] = np.mean(y[bin_indices == nearest_bin])

        # Compute Huber loss
        huber = huber_loss(y, bin_means, delta=delta)
        results.append({"Method": method, "Scenario": scenario, "Bins": n_bins, "Huber Loss": huber})
    
    return pd.DataFrame(results)

# Run evaluation with Huber loss optimization
df_results_huber = pd.concat([
    evaluate_methods_huber("non_monotonic"),
    evaluate_methods_huber("monotonic")
])

df_results_huber

,Method,Scenario,Bins,Huber Loss
0,Decision Tree,non_monotonic,5,3.456610
1,Dynamic Programming,non_monotonic,5,3.456610
2,K-Means,non_monotonic,5,9.042809
3,Isotonic Regression,non_monotonic,5,10.273906
4,Quantile Supervised,non_monotonic,5,9.834877
0,Decision Tree,monotonic,5,20.845927
1,Dynamic Programming,monotonic,5,19.664790
2,K-Means,monotonic,5,20.347413
3,Isotonic Regression,monotonic,5,19.742943
4,Quantile Supervised,monotonic,5,53.599008
